In [ ]:
# _*_ coding: utf-8 _*_
import os
import math
import csv
import pandas as pd
from IPython.display import display
from functools import reduce
import import_ipynb

from CommonModule.Handle_Dir import mkdir_p, del_folder
from CommonModule.ArticleHandler import Article, ArticleReader

In [ ]:
BASE_DIR = "/data/ksb/articles"
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [ ]:
def get_media_name(filepath):
    filename = filepath.split(os.sep)[-1]
    return filename.split(".")[0]

In [ ]:
get_line_token_count = lambda sent : len(sent.split())
get_token_count = lambda sents : reduce(lambda x, y : x + y, map(get_line_token_count, sents))

if __name__ == '__main__':
    
    article_dist = pd.DataFrame(columns=['Title', 'Media', 'Line length', 'Token Number'])
    for idx, media_path in enumerate(iglob(os.path.join(ORIGIN_PATH, '**.csv'), recursive=False)):

        media_name = get_media_name(media_path)
        
        f = open(media_path, 'r', newline="\n", encoding="utf-8")
        rdr = csv.reader(f)
        for [title, contents] in rdr:
            article = Article(title, media_name, contents.split("\t"))
            if not article.content : continue # 본문이 없는 경우를 제외함
            
            contents = list(article.readContent())
            dist= {'Title' : article.title, 'Media' : article.media, 'Line length' : len(contents) , 'Token Number' : get_token_count(contents)}
            article_dist = article_dist.append(dist, ignore_index=True)

In [ ]:
display(pd.DataFrame(article_dist))

In [ ]:
article_dist['Line length'] = article_dist['Line length'].astype('float64')
article_dist['Token Number'] = article_dist['Token Number'].astype('float64')

아래는 수집된 기사의 문장 수 분포를 그래프로 나타낸 것이다.  
대다수의 기사는 20개 이내의 문장 수를 가지는 것을 확인하였다.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

line_len_filter = article_dist['Line length'] <= 80

line_min = np.min(article_dist[line_len_filter]['Line length'])
line_max = np.max(article_dist[line_len_filter]['Line length'])
sns.kdeplot(article_dist[line_len_filter]['Line length'])

plt.rcParams["figure.figsize"] = (14,4)
plt.title("Text Line Count Distribution")
plt.xticks(np.arange(line_min, line_max, step=int(line_max // 15)), \
           ["{}".format(int(x)) for x in np.arange(line_min, line_max, step=int(line_max // 15))])
plt.xlabel('Line Count')
plt.show()

20개 이하 문장 수를 가지는 기사에 한하여 토큰 개수를 그래프로 나타낸 것이다.  
250개 이내의 토큰을 가지는 기사들이 대부분이다.

In [ ]:
line_len_filter = article_dist['Line length'] <= 20

token_min = np.min(article_dist[line_len_filter]['Token Number'])
token_max = np.max(article_dist[line_len_filter]['Token Number'])

sns.kdeplot(article_dist[line_len_filter]['Token Number'])

plt.rcParams["figure.figsize"] = (14,4)
plt.title("Text Line Token Count Distribution")
plt.xticks(np.arange(token_min, token_max, step=int(token_max // 20)), \
           ["{}".format(int(x)) for x in np.arange(token_min, token_max, step=int(token_max // 20))])
plt.xlabel('Token Count')
plt.show()

다음은 `MAX_LINE`에 따라 기사 본문을 분할하는 내용이다.  
각 기사 본문을 40 문장 이하로 분할하여, 문장 수 분포와 토큰의 개수 분포를 살펴본다.

In [ ]:
MAX_LINE = 20

In [ ]:
split_by_max_len = lambda sents : [sents[idx*MAX_LINE : idx*MAX_LINE + MAX_LINE] for idx in range(math.ceil(len(sents) / MAX_LINE))]

if __name__ == '__main__':
    
    article_dist = pd.DataFrame(columns=['Title', 'Media', 'Line length', 'Token Number'])
    for idx, media_path in enumerate(iglob(os.path.join(ORIGIN_PATH, '**.csv'), recursive=False)):

        media_name = get_media_name(media_path)
        
        f = open(media_path, 'r', newline="\n", encoding="utf-8")
        rdr = csv.reader(f)
        for [title, contents] in rdr:
            article = Article(title, media_name, contents.split("\t"))
            if not article.content : continue # 본문이 없는 경우를 제외함
            
            contents = list(article.readContent())
            for idx, cont in enumerate(split_by_max_len(contents)):
                dist= {'Title' : article.title + "--{}".format(idx), 'Media' : article.media, \
                       'Line length' : len(cont) , 'Token Number' : get_token_count(cont)}
                article_dist = article_dist.append(dist, ignore_index=True)

In [ ]:
line_min = np.min(article_dist['Line length'])
line_max = np.max(article_dist['Line length'])

print("Line Count : {line_min} ~ {line_max}".format(line_max=line_max, line_min=line_min))
sns.kdeplot(article_dist['Line length'])

plt.rcParams["figure.figsize"] = (14,4)
plt.title("Text Line Count Distribution")
plt.xticks(np.arange(line_min, line_max, step=int(line_max // 15)), \
           ["{}".format(int(x)) for x in np.arange(line_min, line_max, step=int(line_max // 15))])
plt.xlabel('Line Count')
plt.show()

대다수의 토큰의 개수가 300개 이내임을 확인할 수 있다.  
이 이상의 토큰 수를 가지는 기사를 제외할 것이다.

In [ ]:
token_min = np.min(article_dist['Token Number'])
token_max = np.max(article_dist['Token Number'])

line_len_filter = article_dist['Line length'] <= 40
print("Token Count : {token_min} ~ {token_max}".format(token_min=token_min, token_max=token_max))

sns.kdeplot(article_dist[line_len_filter]['Token Number'])

plt.rcParams["figure.figsize"] = (14,4)
plt.title("Text Line Token Count Distribution")
plt.xticks(np.arange(token_min, token_max, step=int(token_max // 20)), \
           ["{}".format(int(x)) for x in np.arange(token_min, token_max, step=int(token_max // 20))])
plt.xlabel('Token Count')
plt.show()